This layout is created 07/22/2025, aiming to test optomechanical coupling. Grating Couplers are used for coupling laser to devices.
This layout contains:
1. Waveguide test patterns: different coupler parameters and waveguide length.
2. Test pattern from Sauer, V. T. K., Diao, Z., Freeman, M. R. & Hiebert, W. K. Optical racetrack resonator transduction of nanomechanical cantilevers. Nanotechnology 25, 055202 (2014).
3. resonators with different parameter of doubly clamped beam.

In [ ]:
import gdsfactory as gf
from blocks import resonator_with_beam, cross_section_with_sleeves, ring_resonator, cantilever_beam, my_coupler
from functools import partial
total_width = 10
@gf.cell
def device_with_io(device_spec, coupler_spec, cross_section, pitch, lateral_offset=0):
    c = gf.Component()
    device = c << device_spec()
    coupler_left = c << coupler_spec()
    coupler_right = c << coupler_spec()
    coupler_left.dmirror_x()
    coupler_left.dmovex(-pitch/2)
    coupler_right.dmovex(pitch/2)
    device.movey(lateral_offset)
    route_left = gf.routing.route_single(c, coupler_left.ports["o1"], device.ports["o1"],cross_section=cross_section)
    route_right = gf.routing.route_single(c, coupler_right.ports["o1"], device.ports["o2"],cross_section=cross_section)
    return c   
cs = cross_section_with_sleeves(core_width=0.43, total_width=total_width, radius=50)
cs_resonator = cross_section_with_sleeves(core_width=0.43, total_width=total_width, radius=1)
coupler_spec = partial(gf.components.grating_coupler_elliptical, wavelength=1.55,nclad=1,fiber_angle=10, n_periods=60)
gc_spec = partial(my_coupler,coupler=coupler_spec)

In [19]:
@gf.cell
def device_beam_with_resonator(beam_length=10, beam_width=0.13, beam_gap=0.1,
                              resonator_lengthx=5, resonator_radius=10, resoantor_gap=0.1):
    resonator_spec = partial(
        ring_resonator,
        radius=resonator_radius,
        length_y=0,
        length_x=resonator_lengthx,
        gap=resoantor_gap,
        cross_section=cs_resonator,
        total_width=total_width,
        bend="bend_circular",
        x_span = 50
    )
    cantilever_spec = partial(cantilever_beam, length=beam_length, width=beam_width)

    resonator_with_beam_spec = partial(resonator_with_beam, resonator_spec=resonator_spec, cantilever_spec=cantilever_spec, gap=beam_gap)

    c = device_with_io(resonator_with_beam_spec, gc_spec, cross_section=cs, pitch=300)
    return c


In [22]:

c = device_beam_with_resonator(beam_length=3,resonator_radius=5.5)
c.show()

c:\Users\weixi\anaconda3\envs\layout\Lib\site-packages\gdsfactory\components\couplers\coupler90.py:42: UserWarning: {'radius': 5.5} ignored for cross_section 'xs_f245cf25'
  x = gf.get_cross_section(cross_section, radius=radius)


In [7]:
from blocks import grating_coupler_test_block
grating_coupler_test_block(gc_spec=gc_spec, cross_section=cs).show()